In [1]:
!nvidia-smi

Sat Oct  3 05:38:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/resnetplusplus
!ls

Mounted at /content/gdrive
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/resnetplusplus
cloning_script.ipynb  ResUNetplusplus


In [3]:
%cd ./ResUNetplusplus
!ls 

/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/resnetplusplus/ResUNetplusplus
data			infer.py      process_image.py	results.png    unet.py
data_generator.py	metrics.py    __pycache__	Resunet++.png
execution_script.ipynb	m_resunet.py  README.md		resunet.py
files			new_data      result		run.py


In [7]:
# %pycat process_image.py
# %%writefile process_image.py
!python3 process_image.py #- to augment training dataset.
# !python3 data_generator.py
# Total Size:  1000
# Training Size:  800
# Validation Size:  100
# Testing Size:  100
# 100% 100/100 [00:04<00:00, 24.42it/s]
# 100% 100/100 [00:04<00:00, 22.95it/s]
# 100% 800/800 [53:22<00:00,  4.00s/it]

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:04<00:00, 24.42it/s]
100% 100/100 [00:04<00:00, 22.95it/s]
100% 800/800 [53:22<00:00,  4.00s/it]


In [10]:
# %pycat run.py
# %%writefile run.py
!python3 run.py

2020-10-03 07:00:53.346480: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-03 07:00:59.634219: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-03 07:00:59.680984: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-03 07:00:59.681930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-03 07:00:59.681986: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-03 07:00:59.936896: I tensorflow/stream_executor/pl

In [16]:
# %pycat infer.py
# %%writefile infer.py
!python3 infer.py
# https://colab.research.google.com/drive/1TTtNwEF4TjWQoXdvjpWky-dnXN1zFu13

2020-10-03 12:32:53.041006: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-03 12:32:55.652514: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-03 12:32:55.672430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-03 12:32:55.673431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-03 12:32:55.673524: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-03 12:32:55.679343: I tensorflow/stream_executor/pl

In [14]:
%pycat infer.py